In [9]:
!pip install yfinance --quiet

In [10]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [11]:
ticker = 'QQQ'
df = yf.download(ticker, start="2023-01-01", end="2025-07-11", auto_adjust=False)
df.columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']


[*********************100%***********************]  1 of 1 completed


In [ ]:
ticker = 'NVDA'
df = yf.download(ticker, start="2023-01-01", end="2025-07-11", auto_adjust=False)
df.columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

In [ ]:
ticker = 'PSQ'
df = yf.download(ticker, start="2019-01-01", end="2025-07-11", auto_adjust=False)
df.columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']


In [ ]:
ticker = 'DIA'
df = yf.download(ticker, start="2019-01-01", end="2025-07-11", auto_adjust=False)
df.columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']


[*********************100%***********************]  1 of 1 completed


In [12]:
# 📌 2. 기술적 지표 계산
print('ticker =', ticker)


def trend_following_strategy(df):
    df = df.copy()

    # 📌 1. 기술적 지표 계산
    df['SMA20'] = df['Close'].rolling(20).mean()
    df['SMA60'] = df['Close'].rolling(60).mean()
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['EMA5'] = df['Close'].ewm(span=5, adjust=False).mean()
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_GoldenCross'] = (df['MACD'] > df['MACD_signal']) & (df['MACD'].shift(1) <= df['MACD_signal'].shift(1))
    df['RSI14'] = compute_RSI(df['Close'], 14)
    df['Volume_MA5'] = df['Volume'].rolling(5).mean()
    df['Slope_SMA60'] = df['SMA60'].diff(3) / 3

    # 볼린저 밴드
    df['BB_Mid'] = df['Close'].rolling(20).mean()
    df['BB_Std'] = df['Close'].rolling(20).std()
    df['BB_Upper'] = df['BB_Mid'] + 2 * df['BB_Std']
    df['BB_Lower'] = df['BB_Mid'] - 2 * df['BB_Std']

    df["Buy_Signal"] = False
    df["Buy_Reason"] = ""
    df["Sell_Signal"] = False
    df["Sell_Reason"] = ""

    # 📌 2. Buy Logic (Trend Breakout + Bottom Fishing)
    for i in range(3, len(df)):
        row = df.iloc[i]
        reason = ""

        # Trend Breakout
        if (
            row["Slope_SMA60"] > 0 and
            row["Close"] > row["SMA60"] and
            row["MACD_GoldenCross"] and
            row["RSI14"] < 65
        ):
            reason = "Trend Breakout"

        # Bottom Fishing
        elif (
            row["Slope_SMA60"] > 0 and
            row["Close"] < row["BB_Mid"] and
            40 < row["RSI14"] < 60 and
            (row["Close"] - row["BB_Lower"]) / (row["BB_Upper"] - row["BB_Lower"]) < 0.3 and
            row["MACD_GoldenCross"]
        ):
            reason = "Bottom Fishing"

        if reason:
            df.at[df.index[i], "Buy_Signal"] = True
            df.at[df.index[i], "Buy_Reason"] = reason

    # 📌 3. Sell Logic (보수적 추세 약화 기반)
    df["Trailing_High"] = df["Close"].cummax()
    df["Drawdown"] = (df["Close"] - df["Trailing_High"]) / df["Trailing_High"]

    for i in range(3, len(df)):
        row = df.iloc[i]
        reasons = []

        if row["RSI14"] > 70 and row["RSI14"] < df["RSI14"].iloc[i - 1] and row["MACD"] < row["MACD_signal"]:
            reasons.append("Overbought Reversal")

        if row["Close"] < df["SMA20"].iloc[i] * 0.98 and df["SMA20"].iloc[i] < df["SMA20"].iloc[i - 1] and row["RSI14"] > 45:
            reasons.append("SMA20 Breakdown")

        if row["MACD"] < row["MACD_signal"] and row["Close"] < row["EMA5"]:
            reasons.append("Failed Rebound")

        if reasons:
            df.at[df.index[i], "Sell_Signal"] = True
            df.at[df.index[i], "Sell_Reason"] = ", ".join(reasons)

    return df

# RSI 계산 함수
def compute_RSI(series, period=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0).rolling(period).mean()
    loss = -delta.where(delta < 0, 0).rolling(period).mean()
    rs = gain / (loss + 1e-10)
    return 100 - (100 / (1 + rs))


# 📌 4. 거래 기록 생성
trades = []
capital = 10000
holding = False
entry_price = 0
entry_date = None
position = 0
max_close = 0
trailing_stop_pct = 0.3
sell_dates = []
entry_reason = ""

for i in range(len(df)):
    date = df.index[i]
    price = df['Close'].iloc[i]

    if not holding and df["Buy_Signal"].iloc[i]:
        holding = True
        entry_price = price
        entry_date = date
        entry_reason = df["Buy_Reason"].iloc[i]
        position = capital / entry_price
        max_close = price
        buy_index = i  # ← 현재 인덱스 저장

    elif holding:
        max_close = max(max_close, price)

        # Trailing Stop
        if price < max_close * (1 - trailing_stop_pct):
            capital = position * price
            trades.append({
                "Buy Date": entry_date.date(),
                "Buy Price": round(entry_price, 2),
                "Sell Date": date.date(),
                "Sell Price": round(price, 2),
                "Gain (%)": round((price - entry_price) / entry_price * 100, 2),
                "Final Capital ($)": round(capital, 2),
                "Reason": "Trailing Stop"
            })
            holding = False
            continue

        reasons = sell_condition(i, entry_price, buy_index)
        if reasons:
            capital = position * price
            df.at[date, "Sell_Signal"] = True
            df.at[date, "Sell_Reason"] = ", ".join(reasons)

# 매도 시점에 Buy Reason도 함께 기록
            trades.append({
                "Buy Date": entry_date.date(),
                "Buy Price": round(entry_price, 2),
                "Sell Date": date.date(),
                "Sell Price": round(price, 2),
                "Gain (%)": round((price - entry_price) / entry_price * 100, 2),
                "Final Capital ($)": round(capital, 2),
                "Buy Reason": entry_reason,
                "Sell Reason": ", ".join(reasons)
            })
            holding = False

# 📋 결과 출력
trade_df = pd.DataFrame(trades)
print("📋 Swing Trade Records:")
print(trade_df)

# 기간 계산
start_date = df[df["Buy_Signal"] == True].index[0]
end_date = df[df["Sell_Signal"] == True].index[-1]
n_years = (end_date - start_date).days / 365.25

# 초기 자금과 최종 자금
initial_capital = 10000
final_capital = trade_df["Final Capital ($)"].iloc[-1]

# ✅ CAGR 계산
CAGR = (final_capital / initial_capital) ** (1 / n_years) - 1
print(f"CAGR (연평균 수익률): {CAGR * 100:.2f}%")

# ✅ MDD (최대 낙폭) 계산
capital_series = trade_df["Final Capital ($)"].values
peak = capital_series[0]
max_drawdown = 0

for value in capital_series:
    if value > peak:
        peak = value
    drawdown = (peak - value) / peak
    if drawdown > max_drawdown:
        max_drawdown = drawdown

print(f"Max Drawdown (최대 낙폭): {max_drawdown * 100:.2f}%")


# 📊 차트
plt.figure(figsize=(16, 8))
plt.plot(df['Close'], label='Close Price', color='blue')
plt.plot(df['SMA20'], label='SMA20', linestyle='--', color='orange')
plt.plot(df['SMA60'], label='SMA60', linestyle='--', color='green')

# 배경색 (보유 구간)
for _, row in trade_df.iterrows():
    plt.axvspan(pd.to_datetime(row['Buy Date']), pd.to_datetime(row['Sell Date']), color='yellow', alpha=0.4)

plt.plot(df[df["Buy_Signal"]].index, df[df["Buy_Signal"]]['Close'], '^', markersize=10, color='green', label='Buy Signal')
plt.plot(df[df["Sell_Signal"]].index, df[df["Sell_Signal"]]['Close'], 'v', markersize=10, color='red', label='Sell Signal')

plt.title(f"{ticker} Swing Trading Strategy with Reason Labels")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()



ticker = QQQ


KeyError: 'Buy_Signal'